In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time

In [ ]:
# Request the Recipes A-Z page and store the link of each category page in an array
url = 'https://www.allrecipes.com/recipes-a-z-6735880'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
categories = []
for link in soup.find_all('a', {'class': 'link-list__link type--dog-bold type--dog-link'}):
  categories.append(link.get('href'))
print(categories[0:5])  # Print the first 5 links to test whether it worked

In [ ]:
# Now visit every category link and append 21 links from each
# Only limited to 21 since scraping all links from each category page
# could result in over 22000 recipe links!
links = []
count = 0

for i in categories:
  soup = BeautifulSoup(requests.get(i).content, 'html.parser')
  for a in soup.find_all("a", {'class': 'mntl-card-list-items'}):
    links.append(a.get("href"))
    count += 1
    if count > 20:
      break
  count = 0
  sleep_time = np.random.uniform(1,5)
  time.sleep(sleep_time)  # IMPORTANT: Need to wait before sending new request
                          # in order to avoid overloading their server

print(len(links)) # Prints how many recipe links collected

In [ ]:
# Links put in a Pandas series so that it can be filtered for noise later
# For now, raw output saved as CSV to avoid losing data in case runtime disconnects
links_out = pd.Series(links)
df_links = links_out.to_frame(name='URL')
print(df_links)
df_links.to_csv("Recipe_Links_Raw.csv")

In [ ]:
# Some links scraped are not recipe links (articles, blogs, etc)
# Only links with '/recipe/' in the URL are recipes
# Must filter everything else out, including duplicates
links_out_filtered = links_out[(links_out.str.contains("/recipe/")==True)].unique()
print(links_out_filtered.size)

In [ ]:
# Put the filtered series in a Dataframe
links_out_filtered = pd.Series(links_out_filtered)
df_links_filtered = links_out_filtered.to_frame(name='URL')
print(df_links_filtered)

In [ ]:
# Lastly, output filtered results as CSV
df_links_filtered.to_csv('Recipe_Links_Filtered.csv')